In [2]:
import pandas as pd
import numpy as np

def parse_list(string):
    return list(map(int, string.strip("[]").split()))

def calculate_total_value(weights, values, state, capacity):
    total_weight = np.dot(weights, state)
    total_value = np.dot(values, state)
    return total_value if total_weight <= capacity else 0

def genetic_algorithm(weights, values, capacity, population_size, generations, crossover_rate, mutation_rate, patience=100):
    num_items = len(weights)
    population = np.random.randint(2, size=(population_size, num_items))
    best_state = np.zeros(num_items, dtype=int)
    best_value = 0
    no_improvement = 0

    for generation in range(generations):
        fitness_values = np.array([calculate_total_value(weights, values, individual, capacity) for individual in population])
        
        max_fitness = fitness_values.max()
        if max_fitness > best_value:
            best_value = max_fitness
            best_state = population[fitness_values.argmax()].copy()
            no_improvement = 0
        else:
            no_improvement += 1
            if no_improvement > patience:
                break

        if fitness_values.sum() == 0:
            selected_population = population
        else:
            fitness_probabilities = fitness_values / fitness_values.sum()
            selected_indices = np.searchsorted(np.cumsum(fitness_probabilities), np.random.rand(population_size))
            selected_population = population[selected_indices]

        children = np.empty_like(population)
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_population[i], selected_population[i + 1]
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, num_items)
                children[i] = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
                children[i + 1] = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            else:
                children[i] = parent1
                children[i + 1] = parent2

        mutations = np.random.rand(population_size, num_items) < mutation_rate
        children = np.logical_xor(children, mutations).astype(int)
        population = children

    return best_state, best_value

df = pd.read_csv('/mnt/d/Algo code/knapsack_5_items.csv')

overall_best_value = 0
overall_best_state = None
overall_best_row_index = -1

for index, row in df.iterrows():
    weights = np.array(parse_list(row['Weights']))
    values = np.array(parse_list(row['Prices']))
    capacity = int(row['Capacity'])

    population_size = 100
    generations = 1000
    crossover_rate = 0.7
    mutation_rate = 0.01

    best_state, best_value = genetic_algorithm(weights, values, capacity, population_size, generations, crossover_rate, mutation_rate)

    print(f"Row {index}: Best state: {best_state}, Best value: {best_value}")

    if best_value > overall_best_value:
        overall_best_value = best_value
        overall_best_state = best_state
        overall_best_row_index = index

print(f"Overall Best -> Row: {overall_best_row_index}, Best state: {overall_best_state}, Best value: {overall_best_value}")


Row 0: Best state: [0 1 0 0 0], Best value: 19
Row 1: Best state: [1 1 1 1 1], Best value: 47
Row 2: Best state: [1 0 1 0 1], Best value: 36
Row 3: Best state: [1 0 0 0 0], Best value: 19
Row 4: Best state: [0 1 1 1 0], Best value: 44
Row 5: Best state: [1 1 0 1 0], Best value: 48
Row 6: Best state: [0 1 1 0 1], Best value: 37
Row 7: Best state: [0 0 0 0 1], Best value: 13
Row 8: Best state: [1 0 1 1 0], Best value: 35
Row 9: Best state: [1 1 0 1 1], Best value: 45
Row 10: Best state: [1 1 1 1 1], Best value: 44
Row 11: Best state: [1 0 1 1 0], Best value: 43
Row 12: Best state: [1 0 1 0 1], Best value: 30
Row 13: Best state: [0 1 1 0 1], Best value: 38
Row 14: Best state: [0 0 1 0 0], Best value: 12
Row 15: Best state: [1 0 1 0 0], Best value: 18
Row 16: Best state: [0 0 0 1 0], Best value: 19
Row 17: Best state: [0 1 1 1 0], Best value: 34
Row 18: Best state: [1 0 1 1 0], Best value: 43
Row 19: Best state: [1 0 1 0 0], Best value: 35
Row 20: Best state: [1 1 1 0 1], Best value: 65
Ro